In [1]:
import networkx as nx

In [ ]:
with open('out/recipes_list.pickle', 'rb') as f:
    recipes = pickle.load(f)

In [2]:
g_nlg = nx.read_gexf('out/elbulli_nlg.gexf')

In [3]:
g_dat = nx.read_gexf('out/elbulli_dat.gexf')

In [4]:
def get_Techniques_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'tecnica']

def get_TechniqueFamily_Technique(g, t):
    return [k for k in g[t] if g[t][k]['edgetype'] == 'se clasifica']

def get_TechniqueFamilies_Recipe(g, r):
    techs = get_Techniques_Recipe(g, r)
    return [k for t in techs for k in get_Technique_Family(g, t)]

def get_RecipeFamily_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'se clasifica']

def get_Year_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'publicado en']

def get_Temperature_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'temperatura']

def get_Styles_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'estilo']

def get_StyleFamily_Style(g, s):
    return [k for k in g[s] if g[s][k]['edgetype'] == 'se clasifica']

def get_StyleFamilies_Recipe(g, r):
    styles = get_Styles_Recipe(g, r)
    return [k for s in styles for k in get_StyleFamily_Style(g, s)]

def get_Preparations_Recipe(g, r):
    return [k for k in g[r] if g[r][k]['edgetype'] == 'elaboracion']

def get_World_Preparation(g, p):
    return [k for k in g[p] if g[p][k]['edgetype'] == 'mundo']

def get_Worlds_Recipe(g, r):
    preps = get_Preparations_Recipe(g, r)
    return [k for p in preps for k in get_World_Preparation(g, p)]

def get_Ingredients_Preparation(g, p):
    return [k for k in g[p] if g[p][k]['edgetype'] == 'composicion']

def get_Ingredients_Recipe(g, r):
    preps = get_Preparations(g, r)
    return [k for p in preps for k in get_Ingredients_Preparation(g, p)]

def get_Techniques_Preparation(g, p):
    return [k for k in g[p] if g[p][k]['edgetype'] == 'tecnica']

def get_Techniques_Recipe(g, r):
    preps = get_Preparations_Recipe(g, r)
    return [k for p in preps for k in get_Techniques_Preparation(g, p)]

def get_Flavors_Preparation(g, p):
    return [k for k in g[p] if g[p][k]['edgetype'] == 'sabor']

def get_Flavors_Recipe(g, r):
    preps = get_Preparations_Recipe(g, r)
    return [k for p in preps for k in get_Flavor_Preparation(g, p)]

def get_Products_Preparation(g, p):
    prod_rels = ['bañado', 'alcohol', 'chocolate', 'lacteo', 'nuevaPasta', 'producto', 'relleno']
    return [k for k in g[p] if g[p][k]['edgetype'] in prod_rels]

def get_Products_Recipe(g, r):
    preps = get_Preparations_Recipe(g, r)
    return [k for p in preps for k in get_Products_Preparation(p)]

def get_PreparationFamily_Preparation(g, p):
    return [k for k in g[p] if g[p][k]['edgetype'] == 'se clasifica']

def get_PreparationFamilies_Recipe(g, r):
    preps = get_Preparations(g, r)
    return [k for p in preps for k in get_PreparationFamily_Preparation(g, p)]

In [101]:
def get_components_Preparation(g, p):
    prods = get_Products_Preparation(g, p)
    prods = list(map(lambda x: x.replace('Producto:', ''), prods))
    ingrs = get_Ingredients_Preparation(g, p)
    flavs = get_Flavors_Preparation(g, p)
    flavs = list(map(lambda x: x.replace('sabor:', ''), flavs))
    comps = ingrs
    comps.extend(prod for prod in prods if prod not in ingrs)
    comps.extend(flav for flav in flavs if flav not in ingrs)
    return comps

def get_components_Recipe(g, r):
    preps = get_Preparations_Recipe(g, r)
    return [k for p in preps for k in get_components_Preparation(g, p)]

def INGREDIENTS(r):
    ingrs_dat = get_components_Recipe(g_dat, r)
    ingrs_nlg = get_components_Recipe(g_nlg, r)
    ingrs = ingrs_dat
    ingrs.extend(ingr for ingr in ingrs_nlg if ingr not in ingrs_dat)
    return ingrs

def TECHNIQUES(r):
    techs_dat = get_Techniques_Recipe(g_dat, r)
    techs_nlg = get_Techniques_Recipe(g_nlg, r)
    techs = techs_dat
    techs.extend(tech for tech in techs_nlg if tech not in techs_dat)
    return techs

In [102]:
def get_nodes_by_type(g, typ):
    return [n for n, data in g.nodes_iter(data=True) if data['nodetype'] == typ]

def get_recipes(g=g_dat):
    return get_nodes_by_type(g, 'Receta')

In [138]:
g_dat['2']

{'1987': {'edgetype': 'publicado en', 'id': '10230'},
 '2-1': {'edgetype': 'elaboracion', 'id': '10235'},
 '2-2': {'edgetype': 'elaboracion', 'id': '10233'},
 '2-3': {'edgetype': 'elaboracion', 'id': '10231'},
 '2-4': {'edgetype': 'elaboracion', 'id': '10229'},
 '2-5': {'edgetype': 'elaboracion', 'id': '10234'},
 'FRÍA': {'edgetype': 'temperatura', 'id': '10228'},
 'PLATOS': {'edgetype': 'se clasifica', 'id': '10232'}}

In [139]:
get_Techniques_Preparation(g_dat, '2-1')

['jugo']

In [140]:
get_Techniques_Preparation(g_dat, '2-2')

[]

In [141]:
get_Techniques_Preparation(g_dat, '2-3')

['min']

In [142]:
get_Techniques_Preparation(g_dat, '2-4')

['jugo']

In [143]:
get_Techniques_Preparation(g_dat, '2-5')

[]

In [144]:
g_dat.node['2-1']['desc']

'1. Calentar una cuarta parte del jugo y disolver la gelatina, retirar del fuego y mezclar#    con el resto del jugo de trufa.#2. Al mismo tiempo montar la nata en un batidor.#3. Poner a cuajar en la nevera y cuando esté a punto de cuajar añadir poco a poco#    la nata montada hasta homogeneizar ambas texturas. Poner a punto de sal. No #    deben quedar grumos.#4. Rellenar los moldes con la mousse y guardar en la nevera.'

In [126]:
g_nlg['1']

{'1987': {'edgetype': 'publicado en', 'id': '15971'},
 'Elaboracion19871997-523': {'edgetype': 'elaboracion', 'id': '15970'},
 'Elaboracion19871997-542': {'edgetype': 'elaboracion', 'id': '15972'},
 'Estilo19871997-25': {'edgetype': 'estilo', 'id': '15968'},
 'FRÍA': {'edgetype': 'temperatura', 'id': '15969'},
 'PLATOS': {'edgetype': 'se clasifica', 'id': '15967'}}

In [127]:
get_Ingredients_Preparation(g_nlg, 'Elaboracion19871997-523')

['hojas de gelatina', 'oporto', 'agua']

In [128]:
get_Products_Preparation(g_nlg, 'Elaboracion19871997-523')

[]

In [129]:
get_Flavors_Preparation(g_nlg, 'Elaboracion19871997-523')

['sabor:oporto']

In [130]:
get_components_Preparation(g_nlg, 'Elaboracion19871997-523')

['hojas de gelatina', 'oporto', 'agua']

In [131]:
get_Techniques_Preparation(g_nlg, 'Elaboracion19871997-523')

[]

In [132]:
get_Ingredients_Preparation(g_nlg, 'Elaboracion19871997-542')

['melón', 'gelatina de oporto']

In [133]:
get_Products_Preparation(g_nlg, 'Elaboracion19871997-542')

[]

In [134]:
get_Flavors_Preparation(g_nlg, 'Elaboracion19871997-542')

[]

In [135]:
get_components_Preparation(g_nlg, 'Elaboracion19871997-542')

['melón', 'gelatina de oporto']

In [136]:
get_Techniques_Preparation(g_nlg, 'Elaboracion19871997-542')

[]

In [137]:
INGREDIENTS('1')

['melón',
 'hojas de gelatina',
 'oporto',
 'agua',
 'melón',
 'gelatina de oporto',
 'pimienta blanca',
 'aceite de oliva']

In [83]:
TECHNIQUES('1')

[]

In [ ]:
CONTINUAR POR HIRVIENDO-HERVIR (MIRAR QUE HICE EN LA TESIS PARA UNIFICAR - CREO Q ALGO DE REPRESENTANTE-)
SEGUIR PROBANDO QUE LAS FUNCIONES ANTERIORES SON CORRECTAS: BUSCAR NODOS DE TODOS LOS TIPOS Y COMPROBAR
QUE TODO ESTA CORRECTO: HAY PRODUCTOS, SABORES, TECNICAS... EN LAS PREPARACIONES, Y LOS HAY EN LA CARDINALIDAD
CORRECTA.

In [84]:
g_dat['1-1']

{'melón': {'edgetype': 'composicion', 'id': '20182'}}

In [87]:
for n1, n2, data in g_dat.edges(data=True):
    if data['edgetype'] == 'tecnica':
        print(n1, n2, data)
        break

171-3 hirviendo {'edgetype': 'tecnica', 'id': '3584'}


In [44]:
from itertools import *

In [74]:
xx=[]
ll=[1, 2, 3, 4, 5]
for i in range(0, len(ll)+1):
    xx.extend(combinations(ll, i))

In [75]:
len(xx)

32

In [78]:
# xx

In [28]:
from collections import Counter

In [43]:
merging = {}
ll=[]
for r in get_nodes_by_type(g_nlg, 'Tecnica'):
    ll.append(len(get_Technique_Families(r)))
Counter(ll).most_common()

[(1, 139)]

In [77]:
merging = {}
ll=[]
for r in get_recipes():
    ll.append(len(get_Style(r)))
Counter(ll).most_common()

[(0, 590), (1, 423), (2, 147), (3, 42), (4, 10), (5, 1), (12, 1)]

In [35]:
merging = {}
ll=[]
for r in get_recipes():
    ll.append(len(get_Year(r)))
Counter(ll).most_common()

[(1, 1214)]

In [17]:
g_nlg['1']

{'1987': {'edgetype': 'publicado en', 'id': '15971'},
 'Elaboracion19871997-523': {'edgetype': 'elaboracion', 'id': '15970'},
 'Elaboracion19871997-542': {'edgetype': 'elaboracion', 'id': '15972'},
 'Estilo19871997-25': {'edgetype': 'estilo', 'id': '15968'},
 'FRÍA': {'edgetype': 'temperatura', 'id': '15969'},
 'PLATOS': {'edgetype': 'se clasifica', 'id': '15967'}}